In [136]:
import os

import requests
from datetime import datetime, timedelta

In [137]:
WORK_PROJECT_IDS = [186181594, 188079427, 187316243]
TOOLING_PROJECT_IDS = [187944113, 186181587, 188328043]

DATE_FORMAT = "%Y-%m-%d"

LAST_WEEK_FILE_NAME = "/Users/danthompson/Code/Scripts/CLI/toggl textbar/data/total_work_time_last_week.txt"
TODAY_FILE_NAME = "/Users/danthompson/Code/Scripts/CLI/toggl textbar/data/total_work_time_today.txt"

In [138]:

def get_time_entries(start_date, project_ids, end_date=None):
    if end_date is None:
        end_date = datetime.now().strftime(DATE_FORMAT)
    url = "https://api.track.toggl.com/reports/api/v3/workspace/397836/summary/time_entries"
    headers = {
        "Content-Type": "application/json"
    }
    auth = (os.environ.get('TOGGL_API_KEY'), "api_token")
    payload = {
        "start_date": start_date,
        "end_date": end_date,
        "project_ids": project_ids
    }

    response: Response = requests.post(url, headers=headers, auth=auth, json=payload)
    response_json = response.json()
    subgroups = (group['sub_groups'] for group in response_json["groups"])
    nested_subgroups = (subgroup for subgroup in subgroups for subgroup in subgroup)
    return sum([subgroup['seconds'] for subgroup in nested_subgroups])


In [139]:

def get_current_time_entry(project_ids):
    url = "https://api.track.toggl.com/api/v9/me/time_entries/current"
    headers = {
        "Content-Type": "application/json"
    }
    auth = (os.environ.get('TOGGL_API_KEY'), "api_token")

    response = requests.get(url, headers=headers, auth=auth)
    response_json = response.json()

    if not response_json or 'start' not in response_json or "project_id" not in response_json or response_json["project_id"] not in project_ids:
        return None

    return response_json["start"]

In [140]:
def get_time_from_current_time_entry(project_ids):
    start_time_str = get_current_time_entry(project_ids)
    if not start_time_str:
        return 0

    start_time = datetime.strptime(start_time_str, '%Y-%m-%dT%H:%M:%S%z')
    
    return int((datetime.now(start_time.tzinfo) - start_time).total_seconds() / 60)

In [141]:
def get_total_time(project_ids, start_date, current_time, file_name, end_date=None):
    summary = get_time_entries(start_date, project_ids)
    total_mins = current_time + int(summary / 60)
    with open(file_name, "w") as f:
        f.write(str(total_mins))

    return total_mins

In [142]:
def get_last_week_and_today_time(project_ids):
    current_time = get_time_from_current_time_entry(project_ids)
    last_week = (datetime.now() - timedelta(days=6)).strftime(DATE_FORMAT)
    last_week_total_time = get_total_time(project_ids, last_week, current_time, LAST_WEEK_FILE_NAME)
    today = datetime.now().strftime(DATE_FORMAT)
    today_total_time = get_total_time(project_ids, today, current_time, LAST_WEEK_FILE_NAME)
    return last_week_total_time, today_total_time

In [143]:
last_week_total_work_time, today_total_work_time = get_last_week_and_today_time(WORK_PROJECT_IDS)

In [144]:
last_week_total_tooling_time, today_total_tooling_time = get_last_week_and_today_time(TOOLING_PROJECT_IDS)

In [145]:
print(last_week_total_tooling_time)
print(last_week_total_work_time)

844
384


In [146]:
print(f"{last_week_total_work_time//60}:{last_week_total_work_time%60:02d}")
print(f"{today_total_work_time//60}:{today_total_work_time%60:02d}")
print(f"{last_week_total_tooling_time//60}:{last_week_total_tooling_time%60:02d}")
print(f"{today_total_tooling_time//60}:{today_total_tooling_time%60:02d}")

6:24
0:00
14:04
1:46
